In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!mkdir ./dataset
!unzip /content/drive/MyDrive/MLP_final/FaceAge.zip -d ./dataset

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
  inflating: ./dataset/train/49_38550.jpg  
  inflating: ./dataset/train/49_38551.jpg  
  inflating: ./dataset/train/49_38552.jpg  
  inflating: ./dataset/train/49_38553.jpg  
  inflating: ./dataset/train/49_38554.jpg  
  inflating: ./dataset/train/49_38555.jpg  
  inflating: ./dataset/train/49_38556.jpg  
  inflating: ./dataset/train/49_38557.jpg  
  inflating: ./dataset/train/49_38558.jpg  
  inflating: ./dataset/train/49_38559.jpg  
  inflating: ./dataset/train/49_38560.jpg  
  inflating: ./dataset/train/49_38561.jpg  
  inflating: ./dataset/train/49_38562.jpg  
  inflating: ./dataset/train/49_38563.jpg  
  inflating: ./dataset/train/49_38564.jpg  
  inflating: ./dataset/train/49_38565.jpg  
  inflating: ./dataset/train/49_38566.jpg  
  inflating: ./dataset/train/49_38567.jpg  
  inflating: ./dataset/train/49_38568.jpg  
  inflating: ./dataset/train/49_38569.jpg  
  inflating: ./dataset/train/49_38570.jpg  
  inflating: ./dataset/train/49_38571.jp

##**Import all neceassary packages**


In [10]:
import os
import sys
import time
import torch
import numpy as np
from glob import glob
from PIL import Image
import matplotlib.pyplot as plt

import torch.nn as nn
import torch.nn.functional as F
import torch.backends.cudnn as cudnn

from torch.autograd import Variable
from torch.optim.lr_scheduler import MultiStepLR
from torch.utils.data import Dataset
from torchvision import transforms
from tqdm.notebook import tqdm as tqdm


##**Model - Define ResNet Model**

In [11]:
'''ResNet18/34/50/101/152 in Pytorch.'''

def conv3x3(in_planes, out_planes, stride=1):
    return nn.Conv2d(in_planes, out_planes, kernel_size=3, stride=stride, padding=1, bias=False)


class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, in_planes, planes, stride=1):
        super(BasicBlock, self).__init__()
        self.conv1 = conv3x3(in_planes, planes, stride)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = conv3x3(planes, planes)
        self.bn2 = nn.BatchNorm2d(planes)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != self.expansion*planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, self.expansion*planes, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(self.expansion*planes)
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out


class Bottleneck(nn.Module):
    expansion = 4

    def __init__(self, in_planes, planes, stride=1):
        super(Bottleneck, self).__init__()
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)
        self.conv3 = nn.Conv2d(planes, self.expansion*planes, kernel_size=1, bias=False)
        self.bn3 = nn.BatchNorm2d(self.expansion*planes)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != self.expansion*planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, self.expansion*planes, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(self.expansion*planes)
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = F.relu(self.bn2(self.conv2(out)))
        out = self.bn3(self.conv3(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out


class ResNet(nn.Module):
    def __init__(self, block, num_blocks, num_classes=10):
        super(ResNet, self).__init__()
        self.in_planes = 64
        self.bn = nn.BatchNorm2d(3)
        self.conv1 = conv3x3(3,64)
        self.bn1 = nn.BatchNorm2d(64)
        self.layer1 = self._make_layer(block, 64, num_blocks[0], stride=2)
        self.layer2 = self._make_layer(block, 128, num_blocks[1], stride=2)
        self.layer3 = self._make_layer(block, 128, num_blocks[2], stride=2)
        self.linear = nn.Linear(512,num_classes)

    def _make_layer(self, block, planes, num_blocks, stride):
        strides = [stride] + [1]*(num_blocks-1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_planes, planes, stride))
            self.in_planes = planes * block.expansion
        return nn.Sequential(*layers)

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = F.avg_pool2d(out, 4)
        out = out.view(out.size(0), -1)
        out = self.linear(out)
        return torch.clamp(out, min=1.)
        return out


def ResNet18(num_classes=10):
    return ResNet(BasicBlock, [3,3,3,3], num_classes)

def ResNet34(num_classes=10):
    return ResNet(BasicBlock, [3,4,6,3], num_classes)

def ResNet50(num_classes=10):
    return ResNet(Bottleneck, [3,4,6,3], num_classes)

def ResNet101(num_classes=10):
    return ResNet(Bottleneck, [3,4,23,3], num_classes)

def ResNet152(num_classes=10):
    return ResNet(Bottleneck, [3,8,36,3], num_classes)

# test_resnet()

##**Utils**

In [12]:
class AverageMeter(object):
    r"""Computes and stores the average and current value
    """
    def __init__(self, name, fmt=':f'):
        self.name = name
        self.fmt = fmt
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

    def __str__(self):
        fmtstr = '{name} {val' + self.fmt + '} ({avg' + self.fmt + '})'
        return fmtstr.format(**self.__dict__)


class ProgressMeter(object):
    def __init__(self, num_batches, *meters, prefix=""):
        self.batch_fmtstr = self._get_batch_fmtstr(num_batches)
        self.meters = meters
        self.prefix = prefix

    def print(self, batch):
        entries = [self.prefix + self.batch_fmtstr.format(batch)]
        entries += [str(meter) for meter in self.meters]
        print('\t'.join(entries))

    def _get_batch_fmtstr(self, num_batches):
        num_digits = len(str(num_batches // 1))
        fmt = '{:' + str(num_digits) + 'd}'
        return '[' + fmt + '/' + fmt.format(num_batches) + ']'


def accuracy(output, target, topk=(1,)):
    r"""Computes the accuracy over the $k$ top predictions for the specified values of k
    """
    with torch.no_grad():
        maxk = max(topk)
        batch_size = target.size(0)

        # _, pred = output.topk(maxk, 1, True, True)
        # pred = pred.t()
        # correct = pred.eq(target.view(1, -1).expand_as(pred))

        # faster topk (ref: https://github.com/pytorch/pytorch/issues/22812)
        _, idx = output.sort(descending=True)
        pred = idx[:,:maxk]
        pred = pred.t()
        correct = pred.eq(target.view(1, -1).expand_as(pred))

        res = []
        for k in topk:
            correct_k = correct[:k].reshape(-1).float().sum(0, keepdim=True)
            res.append(correct_k.mul_(100.0 / batch_size))
        return res

##**Parameter Settings**

In [13]:
model = 'resnet18' # resnet18, resnet50, resnet101
batch_size = 200  # Input batch size for training (default: 128)
epochs = 400 # Number of epochs to train (default: 100)
learning_rate = 0.0001 # Learning rate
data_augmentation = True # Traditional data augmentation such as augmantation by flipping and cropping?
cutout = True # Apply Cutout?
n_holes = 1 # Number of holes to cut out from image
length = 16 # Length of the holes
num_classes = 1 ##regression
print_freq = 30
num_workers = 1
cuda = torch.cuda.is_available()
cudnn.benchmark = True  # Should make training should go faster for large models
train_val_ratio = 0.8

mean = [0.5754391,  0.45293394, 0.39858824]
std = [0.22728625, 0.20080547, 0.18557058]

model = ResNet18(num_classes=num_classes).cuda()

# Check number of parameters your model
pytorch_total_params = sum(p.numel() for p in model.parameters())
print(f"Number of parameters: {pytorch_total_params}")
if int(pytorch_total_params) > 2000000:
    print('Your model has the number of parameters more than 2 millions..')
    sys.exit()

Number of parameters: 1952455


##**Load and preprocess data**

In [ ]:
class FacialDataset(Dataset):
    def __init__(self, data_path):
        if not os.path.exists(data_path):
            raise Exception(f"[!] {self.data_path} not existed")
        self.imgs = []
        self.labels = []
        self.transform = transforms.Compose([
            transforms.ToTensor(),                                 
            transforms.Resize((64,64))
        ])
        self.age_path = sorted(glob(os.path.join(data_path, "*.*")))
        for pth in self.age_path:
          img = Image.open(pth)
          label = int(pth.split('_')[0].split('/')[-1])
          self.labels.append(label)
          self.imgs.append(self.transform(img))
    
    def __getitem__(self, idx):
        image = self.imgs[idx]
        label = self.labels[idx]
        return image , label

    def __len__(self):
        return len(self.age_path)

def get_data_loader(data_path, batch_size, num_workers,train_val_ratio):
    full_dataset = FacialDataset(data_path)
    train_size = int(train_val_ratio * len(full_dataset))
    val_size = len(full_dataset) - train_size
    train_dataset, val_dataset = torch.utils.data.random_split(full_dataset, [train_size, val_size])
    train_loader = torch.utils.data.DataLoader(dataset=train_dataset,batch_size=batch_size,shuffle=True,num_workers=num_workers)
    val_loader = torch.utils.data.DataLoader(dataset=val_dataset,batch_size=batch_size,shuffle=False,num_workers=num_workers)
    return train_loader, val_loader

train_loader, val_loader = get_data_loader('./dataset/train',batch_size,num_workers,train_val_ratio)


In [ ]:
class Cutout(object):
    """Randomly mask out one or more patches from an image.

    Args:
        n_holes (int): Number of patches to cut out of each image.
        length (int): The length (in pixels) of each square patch.
    """
    def __init__(self, n_holes, length):
        self.n_holes = n_holes
        self.length = length

    def __call__(self, img):
        """
        Args:
            img (Tensor): Tensor image of size (C, H, W).
        Returns:
            Tensor: Image with n_holes of dimension length x length cut out of it.
        """
        h = img.size(2)
        w = img.size(3)

        mask = np.ones((h, w), np.float32)
        
        for n in range(self.n_holes):
            y = np.random.randint(h)
            x = np.random.randint(w)

            y1 = np.clip(y - self.length // 2, 0, h)
            y2 = np.clip(y + self.length // 2, 0, h)
            x1 = np.clip(x - self.length // 2, 0, w)
            x2 = np.clip(x + self.length // 2, 0, w)

            mask[y1: y2, x1: x2] = 0.

        mask = torch.from_numpy(mask)
        mask = mask.expand_as(img)
        img = img * mask
        #plt.imshow(np.transpose(img, (1, 2, 0)))
        #plt.show()
        return img

##**Main Training**

In [ ]:
import math

def train(train_loader, epoch, model, optimizer):
    batch_time = AverageMeter('Time', ':6.3f')
    l2 = AverageMeter('L2 Loss', ':.4e')
    #l1 = AverageMeter('L1 Loss', ':.4e')
    #progress = ProgressMeter(len(train_loader), batch_time, l2,l1, prefix="Epoch: [{}]".format(epoch))
    progress = ProgressMeter(len(train_loader), batch_time, l2, prefix="Epoch: [{}]".format(epoch))
    # switch to train mode
    model.train()
    l1_criterion = nn.L1Loss().cuda()
    l2_criterion = nn.MSELoss().cuda()
    end = time.time()
    total_loss = 0
    transform = transforms.Compose([ # normalization 할까 말까 생각해야돼
            transforms.RandomHorizontalFlip(),
            transforms.RandomRotation((-30, 30)),
            Cutout(n_holes=2, length=16)
    ])
    for i, (input, label) in enumerate(train_loader):
        # measure data loading time
        input = transform(input)
        input = input.cuda()
        label = label.float().flatten().cuda()
        # compute output
        output = model(input).flatten()
        l2_ = l2_criterion(output, label)
        #l1_ = l1_criterion(output,label)
        total_loss += l2_.item()
        l2.update(l2_.item(), input.size(0))
        #l1.update(l1_.item(), input.size(0))
        # compute gradient and do SGD step
        optimizer.zero_grad()
        l2_.backward()
        optimizer.step()

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()

        if i % print_freq == 0:
            progress.print(i)
    print('==> Train Accuracy: Loss {losses:.3f} || scores {r2_score:.4e}'.format(losses=total_loss, r2_score=math.sqrt(total_loss)))

def validation(val_loader,epoch, model):
    model.eval()
    l2 = torch.nn.MSELoss().cuda()
    for i,(input,label) in enumerate(val_loader):
        input = input.cuda()
        label = label.float().flatten().cuda()
        output = model(input)
        output = output.flatten()
        l2_ = l2(label,output)
        RMSE = torch.sqrt(l2_)
    print('==> Validate Accuracy:  L2 distance {:.3f} || RMSE {:.3f}'.format(l2_,RMSE))
    return RMSE

optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=5e-4)
scheduler = MultiStepLR(optimizer, milestones=[40, 80, 120, 160, 200, 240, 280, 320], gamma=0.2)

best_acc = 1e5
for epoch in range(epochs):
    print("\n----- epoch: {}, lr: {} -----".format(
        epoch, optimizer.param_groups[0]["lr"]))

    # train for one epoch
    start_time = time.time()
    train(train_loader, epoch, model, optimizer)
    val_acc = validation(val_loader,epoch,model)

    elapsed_time = time.time() - start_time
    print('==> {:.2f} seconds to train this epoch\n'.format(elapsed_time))
    # learning rate scheduling
    scheduler.step()
    
    # Save model for best accuracy
    if best_acc > val_acc and epoch >= 100:
        best_acc = val_acc
        torch.save(model.state_dict(), './drive/MyDrive/Adam/model_best.pt')
    torch.save(model.state_dict(),'./drive/MyDrive/Adam/model_epoch_{0}_acc_{1}.pt'.format(epoch, val_acc))
print(f"Best RMSE Accuracy: {best_acc}")

## Make an evalutation csv file

This code makes an evaluation csv file for Competition submission.

**Don't change below code!!!**

In [ ]:
class FacialDataset_test(Dataset):
    def __init__(self, data_path):
        if not os.path.exists(data_path):
            raise Exception(f"[!] {self.data_path} not existed")
        self.imgs = []
        self.transform = transforms.Compose([
            transforms.ToTensor(),                                 
            transforms.Resize((64,64)),
        ])
        self.age_path = sorted(glob(os.path.join(data_path, "*.*")))
        for pth in self.age_path:
          img = Image.open(pth)
          self.imgs.append(self.transform(img))
    def __getitem__(self, idx):
        image = self.imgs[idx]
        return image

    def __len__(self):
        return len(self.age_path)

import torch
import pandas as pd
import argparse
import time
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader

def eval():
    test_dataset = FacialDataset_test("./dataset/test")
    test_loader = torch.utils.data.DataLoader(dataset=test_dataset,batch_size=1,shuffle=False)

    model = ResNet18(num_classes=num_classes).cuda()
    model.load_state_dict(torch.load('./drive/MyDrive/Adam/model_best.pt'))
    model.eval()
    print('Make an evaluation csv file for submission...')
    Category = []
    for input in test_loader:
        input = input.cuda()
        output = [model(input).item()]
        # output = torch.argmax(output, dim=1)
        Category = Category + output

    Id = list(range(0, len(test_loader)))
    samples = {
       'Id': Id,
       'Category': Category 
    }
    df = pd.DataFrame(samples, columns=['Id', 'Category'])

    df.to_csv('./drive/MyDrive/Adam/submission_best.csv', index=False)
    print('Done!!')

    del model
    model = ResNet18(num_classes=num_classes).cuda()
    model.load_state_dict(torch.load('./drive/MyDrive/Adam/model_epoch_299_acc_5.480024337768555.pt'))
    #model.load_state_dict(torch.load('./drive/MyDrive/Adam/model_latest.pt'))
    model.eval()
    
    print('Make an evaluation csv file for submission...')
    Category = []
    for input in test_loader:
        input = input.cuda()
        output = [model(input).item()]
        # output = torch.argmax(output, dim=1)
        Category = Category + output

    Id = list(range(0, len(test_loader)))
    samples = {
       'Id': Id,
       'Category': Category 
    }
    df = pd.DataFrame(samples, columns=['Id', 'Category'])

    df.to_csv('./drive/MyDrive/Adam/submission_latest.csv', index=False)
    print('Done!!')

if __name__ == "__main__":
    eval()